In [3]:
def editdf(df,impact,preference,supplement):
    impactperpill = 0.31 # kgCO2eq per pill
    df.at[(df['Food Name'].str.contains('pumpkin'))&(df['Food Name'].str.contains('boiled')),'Food Name'] = 'squash, pumpkin, boiled'
    df.at[(df['Food Name'].str.contains('pumpkin'))&(df['Food Name'].str.contains('raw')),'Food Name'] = 'squash, pumpkin, raw'
    remove = ['AN','OE','AM','BP','AO','SEC','DAP','SN','WCGY','WCG','WCN','Q','SEA','SEC',\
              'SN','WCG','WCN','AA','AA_wg','PAC','SC' ]
    df = df[~df['Group'].isin(remove)]
    df = df[~df['Food Name'].str.contains('ice cream')]
    df = df[~df['Food Name'].str.contains('cocoa')]
    df = df[~df['Food Name'].str.contains('raisins')]
    # remove lemons limes
    sub = df[((df['Food Name'].str.contains('lemon|lime'))&(df['Group']=='FAT'))].index.tolist()
    df.drop(sub,axis=0,inplace=True)
    
    sub1 = df.loc[(df['Food Name'].str.contains('peas|beans|lentils'))&(df['Food Name'].str.contains('raw')\
        &(df['Food Name'].str.contains('dried'))&(~df['Food Name'].str.contains(\
                                'frozen|sugar|runner|cluster|sprouts|canned')))].index.tolist()
    df.drop(sub1,axis=0,inplace=True)
    sub2 = df.loc[(df['Food Name'].str.contains('tomato puree'))].index.tolist()
    df.drop(sub2,axis=0,inplace=True)
    
    df = df.fillna(0)
    df['Food Name']= df[['Food Name','Group']].apply(lambda x: '_'.join(x),axis=1)
    df = df[~((df['Group'].str.startswith('M'))&((df['Group'].str.endswith('R'))))]    
    
    # remove raw meat () and raw fish
    # remove cooked vegetables (DGC)
    #df = df[~(df['Group']=='DGC')]

    df = df[~((df['Group'].str.startswith('J'))&(df['Group'].str.endswith('R')))]
    
    if preference == 'vegan':
        #fish,meat,dairy
        df = df[~(df['Group'].str.startswith('J')|df['Group'].str.startswith('M'))]
        df = df[~(df['Group'].isin(['BLS','BLH','BLF','BN','BAH','BAK','BAE']))]
        df = df[~(df['Group']=='CA')]#butter
        df = df[~(df['Group']=='OA')]#eggs
        df = df[~df['Food Name'].str.contains('egg,')]
        df = df[~df['Food Code'].isin(['11-795','11-793','11-719'])] # get rid of oats with milk and egg noodles
    elif preference == 'vegetarian':
        df = df[~(df['Group'].str.startswith('J')|df['Group'].str.startswith('M'))]
    elif preference == 'pescatarian':
        df = df[~df['Group'].str.startswith('M')]
    else:pass
    
    if supplement == 'yes_sup':
        sup_dict = {'Calcium (mg)':1200,'Vitamin B12 (µg)':3,'Iron (mg)':18,\
        'Zinc (mg)':10,'Selenium (µg)':55,'Retinol Equivalent (µg)':900,'Vitamin D (µg)':20,\
        'Iodine (µg)':150, 'Vitamin K1 (µg)':120,'Food Name':'supplement','Group':'SUP',\
        'optimization_value_GHG_1_trade':impactperpill,'GHG_bioopt_value1':impactperpill,
         'optimization_value_BIO_1_trade':5.5e-16,'bio_GHGopt_value1':5.5e-16,
            'conversion factor':1,'Edible proportion':1,\
                    }     
        df = df.append(sup_dict,ignore_index=True)
        df = df.fillna(0)
        #np.mean(df1['global_landbio_gram'])*10 for value for biodiversity loss from a supplement
    
    else:pass
    # these need to be updated in next version to account for no trade and no home production
    df = df[(df['trade_impacts_GHG'].str.len()!=0)|(df['trade_impacts_BIO'].str.len()!=0)]
    df = df[df['optimization_value_GHG_1_trade']!=0]
    df = df[df['optimization_value_BIO_1_trade']!=0]
    df = df[(df['bio_GHGopt_value1']!=0)|(df['GHG_bioopt_value1']!=0)]

    # additional impact due to not all parts being consumed:
    # take non-edible proportion, multiply by impact per gram of total product, add this to edible proportion impact
    # because even though this isn't consumed as part of the grams recommended, it is still produced. Need to 
    # say non-edible proportion is 1 (not 0) if whole thing is consumed to avoid getting zero impact
    m1 = (df['Edible proportion']!=1)
    df_valid = df[m1]
    df.loc[m1,'Non_edible'] = 1 - df_valid['Edible proportion']
    
    for i in ['optimization_value_GHG_1_trade','bio_GHGopt_value1','optimization_value_BIO_1_trade',\
              'GHG_bioopt_value1']:
        df[i] = df[i]*df['conversion factor']
        df.loc[m1,i] = df.loc[m1,i]+(df.loc[m1,i]*df.loc[m1,'Non_edible'])
    df.reset_index(inplace=True,drop=True)
    return df

In [5]:
#import numpy as np
#import pandas as pd

In [7]:
#df = pd.read_pickle('CH_aug.p')